In [1]:
%matplotlib notebook
import logging
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [2]:
import unittest
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from deepART import ProbART, base, dataset, deepART
from sklearn.datasets import make_moons
from concurrent.futures import ThreadPoolExecutor

/usr/local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Generate dataset

In [3]:
#sample_data = dataset.Clusters2d_overlap(nclusters=6,overlap=0.1,spread=0.1, data_range=[0, 10000])
#sample_data = dataset.TwoSpirals(100, noise=1)

X, y = make_moons(n_samples=200, noise=0.05)
sample_data = dataset.Dataset(X+0.5)

#npoints = 200
#sample_data = dataset.TwoSpirals(npoints, noise=30)

In [4]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)

ax.set_xlabel("X",fontsize=12)
ax.set_ylabel("Y",fontsize=12)
ax.grid(True,linestyle='-',color='0.75')
# scatter with colormap mapping
ax.scatter(sample_data.data_normalized[...,0],sample_data.data_normalized[...,1],s=100,c='k',marker='*')
ax.axis((0, 1, 0, 1))
plt.show()
#fig.savefig('data_raw.png', bbox_inches='tight')

<IPython.core.display.Javascript object>

In [5]:
convergence = 10
neurons = 50
rho = 0.2598
beta = 0.75
alpha = 0.1
main_executor = ThreadPoolExecutor(max_workers=20)
network = deepART.Sequential()
network.add(ProbART( 2, neurons, rho=rho,alpha=alpha, beta=beta,comp=True)) # add first probart layer
network.add(ProbART( 2, neurons, rho=0.211,alpha=alpha, beta=beta,comp=True)) # add second probart layer
print("Start testing...")
predictions = network.fit_predict(epochs=1, x=sample_data.data_normalized, convergence=convergence, shuffle=False, threading=main_executor)
pred_0 = predictions['SubModel_0'] # first unit
pred = predictions['SubModel_1'] # last unit


Start testing...
Epoch: 0
Epoch: 0


In [6]:
pred[0]

[(0, 0.9809310690240026)]

In [7]:
pred_0[0]

[(1, 0.5067173161218216), (0, 0.39539825613907204)]

In [8]:
res = [score for _, score in pred[0]]
idxMax = np.argmax(res)
k, _ = pred[0][idxMax]
k

0

## Generate contour plot for cluster membership


In [9]:
def unpack_results(pred,target):
    #unpack result tuples
    pred_k = []
    scores = []
    data_contour = np.empty((0,2),dtype=np.float32)
    for n, results in enumerate(pred):
        for res_tuple in results:
            k, score = res_tuple
            pred_k.append(k)

            if k == target:
                data_contour = np.vstack((data_contour, sample_data.data_normalized[n]))
                scores.append(score)
    return data_contour, scores, pred_k

In [10]:
def plot_countour(fig,data_contour, scores, sub_index, nplots=(3,2)):
    ax = fig.add_subplot(nplots[0],nplots[1],sub_index)

    ax.set_title("Clustering Results ",fontsize=14)
    ax.set_xlabel("X",fontsize=12)
    ax.set_ylabel("Y",fontsize=12)
    ax.grid(True,linestyle='-',color='0.75')
    # scatter with colormap mapping to predicted class
    ax.tricontour(data_contour[...,0], data_contour[...,1], scores, 14, linewidths=0, colors='k')
    cntr2 = ax.tricontourf(data_contour[...,0], data_contour[...,1], scores, 14, cmap="RdBu_r",)

    fig.colorbar(cntr2, ax=ax)
    ax.plot(data_contour[...,0], data_contour[...,1], 'ko', ms=0.5)
    ax.axis((0, 1, 0, 1))
    ax.set_title('Cluster {}'.format(int(sub_index-1)))

    plt.subplots_adjust(hspace=0.5)
    plt.show()
    #fig.savefig('data_clustered.png', bbox_inches='tight')

In [11]:
#get unique topics
_, _, ks = unpack_results(pred,target=0)
unique_ks = set(ks)
print(unique_ks)

{0, 1}


In [12]:
#plot out clusters memebership
fig = plt.figure(figsize=(8,8))
nplots = (int(np.ceil(len(unique_ks)/2)), 2)
for i in range(len(unique_ks)):
    data_contour, scores, _ = unpack_results(pred,target=i)
    plot_countour(fig, data_contour,scores, sub_index=i+1,nplots=nplots)

<IPython.core.display.Javascript object>

## Cluster segmentation by argmax

In [13]:
def get_clusters(pred):
    ks = []
    for result in pred:
        #unpack prediction
        res = [score for _, score in result]
        idxMax = np.argmax(res)
        k, _ = result[idxMax]
        ks.append(k)
    return ks

In [14]:
ks = get_clusters(pred)

In [15]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)

ax.set_title("Clustering Results ",fontsize=14)
ax.set_xlabel("X",fontsize=12)
ax.set_ylabel("Y",fontsize=12)
ax.set_ylabel("Z",fontsize=12)
ax.grid(True,linestyle='-',color='0.75')
# scatter with colormap mapping to predicted class
color = [c+1 for c in ks]
ax.scatter(sample_data.data_normalized[...,0],sample_data.data_normalized[...,1],s=100,c=color, marker = '*', cmap = cm.jet_r );

plt.show()

<IPython.core.display.Javascript object>

In [16]:
fig.savefig('temp.png', dpi=fig.dpi)

In [17]:
color

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2]

# Fit Method

In [18]:
convergence = 10
neurons = 50
rho = 0.2598
beta = 0.75
alpha = 0.1
main_executor = ThreadPoolExecutor(max_workers=20)
network = deepART.Sequential()
network.add(ProbART( 2, neurons, rho=rho,alpha=alpha, beta=beta,comp=True)) # add first probart layer
network.add(ProbART( 2, neurons, rho=0.211,alpha=alpha, beta=beta,comp=True)) # add second probart layer
network.add(ProbART( 2, neurons, rho=0.211,alpha=alpha, beta=beta,comp=True)) # add third probart layer
print("Start testing...")
network.fit(x=sample_data.data_normalized, epochs=2, shuffle=True, seed=0, threading=main_executor, convergence=convergence, checkpoint=True, every_sample=100, file_path='', return_state=False, start_from=0)


Start testing...
Epoch: 0
Epoch: 1
Epoch: 0
Epoch: 1
Epoch: 0
Epoch: 1


# Predict Method

In [19]:
predictions = network.predict(x=sample_data.data_normalized, threading=main_executor, return_state=True)
pred_0 = predictions['SubModel_0'] # first unit
pred = predictions['SubModel_1'] # second unit

In [20]:
predictions['SubModel_2'] #last unit

[[(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],

# Continue Training Loaded deepART

In [21]:
network = base.load_model('submodel_1_epoch_1_sample_100.p')
network.fit(x=sample_data.data_normalized, epochs=2, shuffle=True, seed=0, threading=main_executor, convergence=convergence, checkpoint=True, every_sample=100, file_path='', return_state=False, start_from=1)
predictions = network.predict(x=sample_data.data_normalized, threading=main_executor, return_state=True)
predictions['SubModel_2'] #last unit

Epoch: 0
Epoch: 1
Epoch: 0
Epoch: 1


[[(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],
 [(0, 1.0)],

# fit_predict method

In [22]:
convergence = 10
neurons = 50
rho = 0.2598
beta = 0.75
alpha = 0.1
main_executor = ThreadPoolExecutor(max_workers=20)
network = deepART.Sequential()
network.add(ProbART( 2, neurons, rho=rho,alpha=alpha, beta=beta,comp=True)) # add first probart layer
network.add(ProbART( 2, neurons, rho=0.211,alpha=alpha, beta=beta,comp=True)) # add second probart layer
network.add(ProbART( 2, neurons, rho=0.211,alpha=alpha, beta=beta,comp=True)) # add third probart layer
print("Start testing...")
predictions = network.fit_predict(x=sample_data.data_normalized, epochs=2, shuffle=True, seed=0, threading=main_executor, convergence=convergence, checkpoint=True, every_sample=100, file_path='')

Start testing...
Epoch: 0
Epoch: 1
Epoch: 0
Epoch: 1
Epoch: 0
Epoch: 1


# Save Method

In [23]:
network.save('mymodel.p')